In [1]:
#vamos usar a versão 3.6.2 do python, pois nessa versão é possível instalar o tf 1.12
import tensorflow as tf
print(tf.__version__)
import json
import numpy as np
import os
import pickle
import logging
logging.basicConfig(format="%(asctime) -15s %(message)s")
import sys
print(sys.version)
import queue
import shutil
import threading
import random
import time

/home/werverton/.pyenv/versions/3.6.2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/werverton/.pyenv/versions/3.6.2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/werverton/.pyenv/versions/3.6.2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/werverton/.pyenv/versions/3.6.2

1.12.0
3.6.2 (default, Aug 31 2021, 22:53:44) 
[GCC 9.3.0]


In [2]:
config_json = {
    "training_data_dir": "/home/werverton/DATA/pickle_files",
    "training_data_dir_test": "/home/werverton/DATA/pickle_files_test",
    "validation_data_dir": "/home/werverton/DATA/validation_data",
    "log_data_dir": "/home/werverton/DATA/training_log",

    "training_dim": 2,
    "predicting_dim": 1,
    "total_interacts": 7,

    "boundary_width": 1,

    "downscale_dim": [180, 320],
    "data_augmentation_threshold": 1,

    "dataset_threads": 100,
    "batch_size": 64,
    "frame_num": 4,

    "learning_rate": 0.01,
    "weight_decay": 0.001,
    "log_step": 200,
    "snapshot_step": 5000,
    "max_iter": 500000,

    "embedding_model": "model_495000.ckpt",
    "embedding_dir": "/home/werverton/DATA/embedding"
}



In [3]:
class Loader():
    """Basic data loader
    """
    def __init__(self, config_json):
        self.x_dim, self.y_dim = config_json["downscale_dim"]
        self.training_dim = config_json["training_dim"]
        self.predicting_dim = config_json["predicting_dim"]
        self.total_interacts = config_json["total_interacts"]
        self.training_data_dir = config_json["training_data_dir"]
        self.logger = logging.getLogger("loader")
        self.logger.setLevel(logging.INFO)

    def next_batch(self):
        pass

In [4]:
class MultipleScreenLoader(Loader):
    """Normal multiple screen data loader, in contrast to debug data loader
    """
    def __init__(self, config_json, load_text=False):
        super().__init__(config_json)
        self.dataset_threads = config_json["dataset_threads"]
        self.batch_size = config_json["batch_size"]
        self.frame_num = config_json["frame_num"]
        self.data_files = next(os.walk(self.training_data_dir))[2]#alterei aqui tirando a variável
        # self.data_files = ["jp.naver.linecard.android.pickle",
        #                    "co.brainly.pickle"]
        self.data_paths = [os.path.join(self.training_data_dir, x) for x in self.data_files]
        self.data_queue = queue.Queue()
        self.path_queue = queue.Queue()
        self.epochs = -1
        self.loading_thread = None
        self.loading_thread_result = None
        self.loading_thread_out = None
        self.produce_threshold = 10000
        self.stopped = False
        self.load_text = load_text

    def get_current_epoch(self):
        return self.epochs

    def reload_paths(self):
        self.epochs += 1
        self.logger.info("epoch: %d", self.epochs)
        random.shuffle(self.data_paths)
        for data_path in self.data_paths:
            self.path_queue.put(data_path)

    def load_pickles(self, data_paths):
        data_item_list = []
        for data_path in data_paths:
            with open(data_path, "rb") as f:
                input_data = pickle.load(f)
            # assemble frames
            # do zero padding before the first image
            for trace_key in input_data:
                image_num = len(input_data[trace_key])
                
                if image_num == 0:
                    continue
                stacked_images = np.stack([np.zeros_like(input_data[trace_key][0][0], dtype=np.float32)] * (self.frame_num - 1) + \
                                          [x[0] for x in input_data[trace_key]], axis=0)
                print("stacked: ",stacked_images)
                images = [stacked_images[i:i + self.frame_num].copy() for i in range(image_num)]
                #print(type(images))
                #print(images)
                # for each image, clear last heatmaps
                for image in images:
                    #print("one image: ",image)
                    image[self.frame_num - 1, :, :, -self.predicting_dim:] = 0.0
                    image -= 0.5
                    # for i in range(self.frame_num):
                    #     visualize_data(image[i])

                heatmaps = [x[0][:,:,-self.predicting_dim:].copy() for x in input_data[trace_key]]
                interacts = np.split(np.eye(self.total_interacts)[[x[1]["interact_type"] for x in input_data[trace_key]]],
                                     image_num, axis=0)

                if self.load_text:
                    input_texts = []
                    for x in input_data[trace_key]:
                        if "text" in x[1]:
                            input_texts.append(x[1]["text"])
                        else:
                            input_texts.append(None)
                    zipped_data = zip(images, heatmaps, interacts, input_texts)
                else:
                    zipped_data = zip(images, heatmaps, interacts)

                for data_item in zipped_data:
                    data_item_list.append(data_item)
        rand_idx = list(range(len(data_item_list)))
        random.shuffle(rand_idx)
        for i in rand_idx:
            self.data_queue.put(data_item_list[i])

    def next_batch_producer(self):
        # always try to load data when < threshold
        # poll check threshold
        while not self.stopped:
            if self.data_queue.qsize() < self.produce_threshold:
                paths_to_load = []
                for i in range(min(self.dataset_threads, len(self.data_paths))):
                    if self.path_queue.empty():
                        self.reload_paths()
                    paths_to_load.append(self.path_queue.get())
                    # self.logger.info("loading: %s", paths_to_load[-1])
                self.load_pickles(paths_to_load)
            time.sleep(1)

    def next_batch_consumer(self):
        # always try to get data
        batch_image_list = []
        batch_heatmap_list = []
        batch_interact_list = []
        if self.load_text:
            batch_input_text_list = []
        for i in range(self.batch_size):
            if self.load_text:
                image, heatmap, interact, input_text = self.data_queue.get()
            else:
                image, heatmap, interact = self.data_queue.get()
            batch_image_list.append(image)
            batch_heatmap_list.append(heatmap)
            batch_interact_list.append(interact)
            if self.load_text:
                batch_input_text_list.append(input_text)
        if self.load_text:
            return (np.concatenate(batch_image_list, axis=0), \
                    np.stack(batch_heatmap_list, axis=0), \
                    np.concatenate(batch_interact_list, axis=0), \
                    batch_input_text_list)
        else:
            return (np.concatenate(batch_image_list, axis=0), \
                    np.stack(batch_heatmap_list, axis=0), \
                    np.concatenate(batch_interact_list, axis=0))

    def next_batch(self):
        if self.loading_thread is None:
            self.loading_thread = threading.Thread(target=self.next_batch_producer)
            self.loading_thread.start()
        return self.next_batch_consumer()

    def stop(self):
        self.stopped = True

In [5]:
class BaseModel():
    """Base model
       Build CNN and LSTM units
    """
    def __init__(self, config_json, training=True):
        self.x_dim, self.y_dim = config_json["downscale_dim"]
        self.training_dim = config_json["training_dim"]
        self.predicting_dim = config_json["predicting_dim"]
        self.total_channels = self.training_dim + self.predicting_dim
        self.total_interacts = config_json["total_interacts"]
        if training:
            self.weight_decay = config_json["weight_decay"]
            self.batch_size = config_json["batch_size"]
            self.keep_prob = 0.5
        else:
            self.weight_decay = 0.0
            self.batch_size = 1
            self.keep_prob = 1.0
        self.regularizer = tf.contrib.layers.l2_regularizer(scale=self.weight_decay)

        self.input_images = None
        self.true_heats = tf.placeholder(dtype=tf.float32,
                                         shape=(None, self.x_dim, self.y_dim, self.predicting_dim))
        self.true_interacts = tf.placeholder(dtype=tf.float32,
                                             shape=(None, self.total_interacts))

        # assign later
        self.heatmap_out = None
        self.interact_out = None

    def get_feed_dict(self, images, heatmaps, interacts):
        return {
            self.input_images: images,
            self.true_heats: heatmaps,
            self.true_interacts: interacts
        }

    def build_cnn(self):
        # First generate low-resolution heatmap
        # 180x320
        self.conv1 = tf.layers.conv2d(inputs=self.input_images,
                                      filters=16,
                                      kernel_size=3,
                                      padding="same",
                                      activation=tf.nn.relu,
                                      kernel_regularizer=self.regularizer,
                                      bias_regularizer=self.regularizer,
                                      name="conv1")
        self.pool1 = tf.layers.max_pooling2d(inputs=self.conv1,
                                             pool_size=2,
                                             strides=2,
                                             name="pool1")
        # 90x160
        self.conv2 = tf.layers.conv2d(inputs=self.pool1,
                                      filters=32,
                                      kernel_size=3,
                                      padding="same",
                                      activation=tf.nn.relu,
                                      kernel_regularizer=self.regularizer,
                                      bias_regularizer=self.regularizer,
                                      name="conv2")
        self.pool2 = tf.layers.max_pooling2d(inputs=self.conv2,
                                             pool_size=2,
                                             strides=2,
                                             name="pool2")
        # 45x80
        self.conv3 = tf.layers.conv2d(inputs=self.pool2,
                                      filters=64,
                                      kernel_size=3,
                                      padding="same",
                                      activation=tf.nn.relu,
                                      kernel_regularizer=self.regularizer,
                                      bias_regularizer=self.regularizer,
                                      name="conv3")
        self.pool3 = tf.layers.max_pooling2d(inputs=self.conv3,
                                             pool_size=2,
                                             strides=2,
                                             padding="same",
                                             name="pool3")
        # 23x40
        self.conv4 = tf.layers.conv2d(inputs=self.pool3,
                                      filters=64,
                                      kernel_size=3,
                                      padding="same",
                                      activation=tf.nn.relu,
                                      kernel_regularizer=self.regularizer,
                                      bias_regularizer=self.regularizer,
                                      name="conv4")
        self.pool4 = tf.layers.max_pooling2d(inputs=self.conv4,
                                             pool_size=2,
                                             strides=2,
                                             padding="same",
                                             name="pool4")
        # 12x20
        self.conv5 = tf.layers.conv2d(inputs=self.pool4,
                                      filters=64,
                                      kernel_size=3,
                                      padding="same",
                                      activation=tf.nn.relu,
                                      kernel_regularizer=self.regularizer,
                                      bias_regularizer=self.regularizer,
                                      name="conv5")
        self.pool5 = tf.layers.max_pooling2d(inputs=self.conv5,
                                             pool_size=2,
                                             strides=2,
                                             padding="same",
                                             name="pool5")
        # 6x10

        # generate heats, i.e. single channel output
        self.pool3_heat = tf.layers.conv2d(inputs=self.pool3,
                                           filters=1,
                                           kernel_size=1,
                                           padding="same",
                                           activation=tf.nn.relu,
                                           kernel_regularizer=self.regularizer,
                                           bias_regularizer=self.regularizer,
                                           name="pool3_heat")
        self.pool4_heat = tf.layers.conv2d(inputs=self.pool4,
                                           filters=1,
                                           kernel_size=1,
                                           padding="same",
                                           activation=tf.nn.relu,
                                           kernel_regularizer=self.regularizer,
                                           bias_regularizer=self.regularizer,
                                           name="pool4_heat")
        self.pool5_heat = tf.layers.conv2d(inputs=self.pool5,
                                           filters=1,
                                           kernel_size=1,
                                           padding="same",
                                           activation=tf.nn.relu,
                                           kernel_regularizer=self.regularizer,
                                           bias_regularizer=self.regularizer,
                                           name="pool5_heat")

    def build_loss(self):
        # heatmap loss
        self.heatmap_loss = tf.losses.softmax_cross_entropy(
            tf.reshape(self.true_heats, [-1, 180 * 320]),
            tf.reshape(self.heatmap_out, [-1, 180 * 320]))
        self.predict_heatmaps = tf.reshape(tf.nn.softmax(
            tf.reshape(self.heatmap_out, [-1, 180 * 320])), [-1, 180, 320, 1])
        # interact loss
        self.interact_out_flat = tf.reshape(self.interact_out, [-1, 6 * 10 * 1])
        self.fc = tf.layers.dense(self.interact_out_flat,
                                  self.total_interacts,
                                  activation=tf.nn.relu,
                                  kernel_regularizer=self.regularizer,
                                  bias_regularizer=self.regularizer,
                                  name="fc")
        # self.fc_dropout = tf.layers.dropout(self.fc, name="fc_dropout")
        self.interact_loss = tf.losses.softmax_cross_entropy(self.true_interacts,
                                                             self.fc)
                                                             # self.fc_dropout)
        self.predict_interacts = tf.nn.softmax(self.fc)

        # total loss
        # self.total_loss = tf.add(self.heatmap_loss, self.interact_loss, name="total_loss")
        # self.total_loss = self.heatmap_loss
        self.total_loss = tf.losses.get_total_loss()

In [6]:
class MultipleScreenModel(BaseModel):
    """Model for processing single screenshot
       Use conv-pool-de-conv for heatmap
       Use conv-pool-fc for predicting

       input: batch_num, x_dim, y_dim, channels
    """
    def __init__(self, config_json, training=True):
        super().__init__(config_json, training)
        self.frame_num = config_json["frame_num"]
        self.input_images = tf.placeholder(dtype=tf.float32,
                                           shape=(None, self.x_dim, self.y_dim,
                                                  self.training_dim + self.predicting_dim))
        self.build_cnn()
        self.build_model()
        self.build_loss()
        if training:
            self.build_summary()

    def build_model(self):
        # generate heats
        # using three LSTMs at different resolutions
        # from pool3, pool4 and pool5

        # pool3_heat_in: item_num (how many series), frame_num, x_dim * y_dim
        self.pool3_heat_in = tf.reshape(self.pool3_heat, [-1, self.frame_num, 23 * 40])
        self.pool4_heat_in = tf.reshape(self.pool4_heat, [-1, self.frame_num, 12 * 20])
        self.pool5_heat_in = tf.reshape(self.pool5_heat, [-1, self.frame_num, 6 * 10])

        # pool3_heat_out: item_num (how many series), x_dim, y_dim
        self.pool3_heat_out = tf.add(
            tf.reshape(
            tf.keras.layers.LSTM(units=23 * 40, dropout=self.keep_prob)(self.pool3_heat_in),
            # tf.keras.layers.LSTM(units=23 * 40)(self.pool3_heat_in),
            [-1, 23, 40, 1]),
            tf.reshape(self.pool3_heat,
            [self.batch_size, self.frame_num, 23, 40, 1])[:, self.frame_num - 1, :,:,:])
        self.pool4_heat_out = tf.add(tf.reshape(
            tf.keras.layers.LSTM(units=12 * 20, dropout=self.keep_prob)(self.pool4_heat_in),
            # tf.keras.layers.LSTM(units=12 * 20)(self.pool4_heat_in),
            [-1, 12, 20, 1]),
            tf.reshape(self.pool4_heat,
            [self.batch_size, self.frame_num, 12, 20, 1])[:, self.frame_num - 1, :,:,:])
        self.pool5_heat_out = tf.add(tf.reshape(
            tf.keras.layers.LSTM(units=6 * 10, dropout=self.keep_prob)(self.pool5_heat_in),
            # tf.keras.layers.LSTM(units=6 * 10)(self.pool5_heat_in),
            [-1, 6, 10, 1]),
            tf.reshape(self.pool5_heat,
            [self.batch_size, self.frame_num, 6, 10, 1])[:, self.frame_num - 1, :,:,:])

        # do upsampling
        # 6x10
        self.pool5_up_filters = tf.get_variable("pool5_up_filters", [4, 4, 1, 1], regularizer=self.regularizer)
        self.pool5_up = tf.nn.relu(tf.nn.conv2d_transpose(value=self.pool5_heat_out,
                                   filter=self.pool5_up_filters,
                                   output_shape=[self.batch_size, 12, 20, 1],
                                   strides=[1, 2, 2, 1],
                                   name="pool5_up"))
        # 12x20
        self.pool4_heat_sum = tf.add(self.pool5_up, self.pool4_heat_out, name="pool4_heat_sum")
        self.pool4_up_filters = tf.get_variable("pool4_up_filters", [4, 4, 1, 1], regularizer=self.regularizer)
        self.pool4_up = tf.nn.relu(tf.nn.conv2d_transpose(value=self.pool4_heat_sum,
                                   filter=self.pool4_up_filters,
                                   output_shape=[self.batch_size, 23, 40, 1],
                                   strides=[1, 2, 2, 1],
                                   name="pool4_up"))
        # 23x40
        self.pool3_heat_sum = tf.add(self.pool4_up, self.pool3_heat_out, name="pool3_heat_sum")
        self.pool3_up_filters = tf.get_variable("pool3_up_filters", [16, 16, 1, 1], regularizer=self.regularizer)
        self.pool3_up = tf.nn.relu(tf.nn.conv2d_transpose(value=self.pool3_heat_sum,
                                   filter=self.pool3_up_filters,
                                   output_shape=[self.batch_size, 180, 320, 1],
                                   strides=[1, 8, 8, 1],
                                   name="pool3_up"))
        self.heatmap_out = self.pool3_up
        self.interact_out = self.pool5_heat_out

    def build_summary(self):
        # summary
        tf.summary.scalar("heatmap_loss", self.heatmap_loss)
        tf.summary.scalar("interact_loss", self.interact_loss)
        tf.summary.scalar("total_loss", self.total_loss)
        tf.summary.image("input_images",
                         self.input_images,
                         max_outputs=self.batch_size*self.frame_num)
        """
        tf.summary.image("sample_heatmaps",
                         self.input_images[:,:,:,-self.predicting_dim:],
                         max_outputs=self.batch_size*self.frame_num)
        """
        tf.summary.image("true_heatmaps",
                         self.true_heats,
                         max_outputs=self.batch_size)
        tf.summary.image("predict_heatmaps",
                         self.predict_heatmaps,
                         max_outputs=self.batch_size)
        tf.summary.histogram("true_interacts", self.true_interacts)
        tf.summary.histogram("predict_interacts", self.predict_interacts)

        tf.summary.histogram("conv1_activation", self.conv1)
        tf.summary.histogram("conv2_activation", self.conv2)
        tf.summary.histogram("conv3_activation", self.conv3)
        tf.summary.histogram("conv4_activation", self.conv4)
        tf.summary.histogram("conv5_activation", self.conv5)
        tf.summary.histogram("conv1_gradient", tf.gradients(self.total_loss, self.conv1))
        tf.summary.histogram("conv2_gradient", tf.gradients(self.total_loss, self.conv2))
        tf.summary.histogram("conv3_gradient", tf.gradients(self.total_loss, self.conv3))
        tf.summary.histogram("conv4_gradient", tf.gradients(self.total_loss, self.conv4))
        tf.summary.histogram("conv5_gradient", tf.gradients(self.total_loss, self.conv5))

        tf.summary.histogram("pool3_heat_out_activation", self.pool3_heat_out)
        tf.summary.histogram("pool4_heat_out_activation", self.pool4_heat_out)
        tf.summary.histogram("pool5_heat_out_activation", self.pool5_heat_out)
        tf.summary.histogram("pool3_heat_out_gradient", tf.gradients(self.total_loss, self.pool3_heat_out))
        tf.summary.histogram("pool4_heat_out_gradient", tf.gradients(self.total_loss, self.pool4_heat_out))
        tf.summary.histogram("pool5_heat_out_gradient", tf.gradients(self.total_loss, self.pool5_heat_out))

        tf.summary.histogram("pool3_up_filters_data", self.pool3_up_filters)
        tf.summary.histogram("pool4_up_filters_data", self.pool4_up_filters)
        tf.summary.histogram("pool5_up_filters_data", self.pool5_up_filters)
        tf.summary.histogram("pool3_up_filters_gradient", tf.gradients(self.total_loss, self.pool3_up_filters))
        tf.summary.histogram("pool4_up_filters_gradient", tf.gradients(self.total_loss, self.pool4_up_filters))
        tf.summary.histogram("pool5_up_filters_gradient", tf.gradients(self.total_loss, self.pool5_up_filters))


In [7]:
log_data_dir = config_json["log_data_dir"]
shutil.rmtree(log_data_dir)
os.makedirs(log_data_dir)

learning_rate = config_json["learning_rate"]
max_iter = config_json["max_iter"]
log_step = config_json["log_step"]
snapshot_step = config_json["snapshot_step"]


In [8]:
data_loader = MultipleScreenLoader(config_json)
model = MultipleScreenModel(config_json)

In [9]:
merged_summary = tf.summary.merge_all()

In [10]:
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth= True

In [11]:
logger = logging.getLogger("train")
logger.setLevel(logging.INFO)

In [12]:
saver = tf.train.Saver(max_to_keep=None)

In [13]:
with tf.Session(config=tf_config) as sess:
    train_writer = tf.summary.FileWriter(log_data_dir, sess.graph)

    final_optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9)

    final_trainer = final_optimizer.minimize(model.total_loss)

    sess.run(tf.global_variables_initializer())

    for i in range(max_iter):
        feed_dict = model.get_feed_dict(*data_loader.next_batch())

        sess.run(final_trainer, feed_dict=feed_dict)

        if i % snapshot_step == 0:
            saved_path = saver.save(sess, os.path.join(log_data_dir, "model_%d.ckpt" % i))
            logger.info("model saved in path: %s" % saved_path)
        
        if i % log_step ==0:
            summary = sess.run(merged_summary, feed_dict=feed_dict)
            train_writer.add_summary(summary, i)
            train_writer.flush()
data_loader.stop()


KeyboardInterrupt: 

ModuleNotFoundError: No module named 'tensorboard.notebook'

4

TypeError: list indices must be integers or slices, not tuple

list

array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.]])

NameError: name 'image_num' is not defined

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 4)

TypeError: string indices must be integers

In [ ]:
heatmaps = np.stack([x[0][:,:,-self.predicting_dim:] for x in input_data["trace_0"]], axis=0)
    
interacts = np.eye(self.total_interacts)[[x[1]["interact_type"] for x in input_data["trace_0"]]]
        

<class '__main__.DebugSingleScreenLoader'>


[[1, 2, 3]]

['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/13.json', {'interact_type': 0}, (168, 19)]


[array([[ 0.11491497, -0.5510944 , -1.9951611 ,  0.82516401],
        [-0.19020027, -0.64225303,  0.13209974, -0.46642741],
        [-0.61126177,  0.34230667, -0.24379606,  2.19217689]]),
 array([[ 0.61098618,  2.30553573,  0.56500586, -1.53850315],
        [-0.85236358, -0.80890938,  0.5498686 ,  2.32425378],
        [ 1.59278725,  0.92651452, -1.05332817, -0.04915212]]),
 array([[-0.44887249, -0.9834613 ,  0.78494405, -1.71417605],
        [ 0.22985371, -0.52957089, -0.11958922,  0.19680375],
        [ 0.6136462 , -1.61421579,  0.3229461 , -0.29113052]]),
 array([[-1.33976936, -0.13098723, -0.17239219, -0.19595837],
        [ 0.1859943 , -1.08775161,  1.18169618, -1.82527334],
        [ 0.96715911, -0.94447868, -0.21973547, -1.92609474]]),
 array([[ 0.24748636,  0.81652946, -0.11550025, -0.00345217],
        [ 0.68164848,  0.44206775, -0.6146099 ,  1.3465075 ],
        [ 2.12580138,  0.57399974, -1.21397099,  0.02364547]]),
 array([[-0.17962378, -0.87802314, -0.13126707,  0.21478895]

In [ ]:
images = np.stack(arrays, axis=0).shape

In [ ]:
print(images)

(10, 3, 4)


In [ ]:
a = [['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/5.json', {'interact_type': 0}, (165, 102)],
['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/13.json', {'interact_type': 0}, (168, 19)]]

In [ ]:
array_a = np.array(a)

/home/werverton/.pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
print(array_a)

[['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/5.json'
  {'interact_type': 0} (165, 102)]
 ['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/13.json'
  {'interact_type': 0} (168, 19)]]


In [ ]:
print(array_a.shape)

(2, 3)


In [ ]:
images = np.stack(array_a, axis=0)

In [ ]:
images.shape

(2, 3)

In [ ]:
images

array([['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/5.json',
        {'interact_type': 0}, (165, 102)],
       ['/home/werverton/DATA/traces/filtered_traces/jp.naver.linecard.android/trace_0/view_hierarchies/13.json',
        {'interact_type': 0}, (168, 19)]], dtype=object)

In [ ]:
np_2d = np.array([[[1.73, 1.68, 1.71, 1.89, 1.79],
                  [65.4, 59.2, 63.6, 88.4,68.7],
                  [1, 2 , 3, 4, 5 ]]])

In [ ]:
np_2d[0,0]

array([1.73, 1.68, 1.71, 1.89, 1.79])